# Event Attribution Analysis - Production Version
● Here are the 4 functions you can use immediately:

  1. find_significant_days(ticker, start_date=None, end_date=None, abs_threshold=2.0, rel_threshold=2.0, limit=None)
    - Finds days with big price movements
  2. get_attribution_events(ticker, impact_date, hourly_threshold=2.0, session_threshold=2.0, daily_threshold=2.0, debug=False)
    - Gets events that contributed to a specific day's return
  3. prepare_attribution_package(ticker, impact_date, hourly_threshold=2.0, session_threshold=2.0, daily_threshold=2.0)
    - Returns complete data package with all event details
  4. prepare_attribution_references(ticker, impact_date, hourly_threshold=2.0, session_threshold=2.0, daily_threshold=2.0)
    - Returns lightweight node IDs for MCP fetching

This notebook provides functions to attribute stock price movements to events (News, Transcripts, Reports) while properly handling:
- 4pm-to-4pm daily return windows
- Weekends and holidays
- Post-market events affecting next trading day
- Multi-threshold filtering for different return types

In [1]:
# Imports
import sys
import os
import pandas as pd
from datetime import datetime, timedelta
import json

# Add project root to path
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))

# Set Neo4j connection environment variables
os.environ['NEO4J_URI'] = 'bolt://localhost:30687'
os.environ['NEO4J_USERNAME'] = 'neo4j'
os.environ['NEO4J_PASSWORD'] = 'Next2020#'

from neograph.Neo4jConnection import get_manager

## Core Functions

In [2]:
def find_significant_days(ticker: str, start_date: str = None, end_date: str = None,
                         abs_threshold: float = 2.0, rel_threshold: float = 2.0,
                         limit: int = None):
    """
    Find days with significant price movements.
    Uses HAS_PRICE for accurate 4pm-to-4pm returns.
    
    Parameters:
    -----------
    ticker: Company ticker
    start_date: Optional start date (YYYY-MM-DD). If None, searches from beginning
    end_date: Optional end date (YYYY-MM-DD). If None, searches to present
    abs_threshold: Minimum absolute stock return % (default 2%)
    rel_threshold: Minimum adjusted return % (stock - market) (default 2%)
    limit: Optional limit on results. If None, returns all matching days
    """
    
    # Build query with optional date filters
    query = """
    MATCH (d:Date)-[hp:HAS_PRICE]->(c:Company {ticker: $ticker})
    MATCH (d)-[hp_spy:HAS_PRICE]->(spy:MarketIndex {ticker: 'SPY'})
    WHERE abs(hp.daily_return) >= $abs_threshold
      AND abs(hp.daily_return - hp_spy.daily_return) >= $rel_threshold
      AND ($start_date IS NULL OR d.date >= $start_date)
      AND ($end_date IS NULL OR d.date <= $end_date)
    RETURN d.date as date,
           hp.daily_return as stock_return,
           hp_spy.daily_return as market_return,
           round(hp.daily_return - hp_spy.daily_return, 2) as adjusted_return,
           hp.volume as volume
    ORDER BY abs(hp.daily_return - hp_spy.daily_return) DESC
    """
    
    # Add limit if specified
    if limit:
        query += f" LIMIT {limit}"
    
    manager = get_manager()
    results = manager.execute_cypher_query_all(
        query,
        parameters={
            "ticker": ticker,
            "start_date": start_date,
            "end_date": end_date,
            "abs_threshold": abs_threshold,
            "rel_threshold": rel_threshold
        }
    )
    
    df = pd.DataFrame(results)
    
    # Calculate day of week after fetching
    if not df.empty:
        df['date_obj'] = pd.to_datetime(df['date'])
        df['day_of_week'] = df['date_obj'].dt.day_name()
    
    return df

In [3]:
def get_attribution_events(ticker: str, impact_date: str, 
                          hourly_threshold: float = 2.0,
                          session_threshold: float = 2.0,
                          daily_threshold: float = 2.0,
                          debug: bool = False):
    """
    Get all events that contributed to a specific date's return.
    Properly handles weekends, holidays, and market sessions.
    
    Parameters:
    -----------
    ticker: Company ticker
    impact_date: The date whose return we're analyzing (YYYY-MM-DD)
    hourly_threshold: Min adjusted return for hourly (default 2%)
    session_threshold: Min adjusted return for session (default 2%)
    daily_threshold: Min adjusted return for daily (default 2%)
    debug: Show debug information
    """
    
    manager = get_manager()
    
    # Get events with richer content for LangGraph
    events_query = """
    MATCH (impact:Date {date: $impact_date})
    MATCH (c:Company {ticker: $ticker})
    
    // Get previous trading days (handle weekends/holidays)
    OPTIONAL MATCH path = (d1:Date)-[:NEXT*1..5]->(impact)
    WHERE d1.is_trading_day = '1'
    
    WITH impact, c, collect(DISTINCT d1) + [impact] as check_dates
    
    // Find events on these dates
    UNWIND check_dates as check_date
    MATCH (event)-[rel:INFLUENCES|PRIMARY_FILER]->(c)
    WHERE (event:News OR event:Transcript OR event:Report)
      AND date(datetime(event.created)) = date(check_date.date)
      AND (
        // Post-market events from previous days
        (check_date.date < $impact_date AND event.market_session = 'post_market')
        OR
        // Pre-market and in-market from impact date
        (check_date.date = $impact_date AND event.market_session IN ['pre_market', 'in_market'])
      )
      // Check if ANY return type meets its threshold
      AND (
        abs(coalesce(rel.hourly_stock, 0) - coalesce(rel.hourly_macro, 0)) >= $hourly_threshold
        OR abs(coalesce(rel.session_stock, 0) - coalesce(rel.session_macro, 0)) >= $session_threshold
        OR abs(coalesce(rel.daily_stock, 0) - coalesce(rel.daily_macro, 0)) >= $daily_threshold
      )
    
    WITH event, rel, check_date, impact, c,
         duration.between(date(check_date.date), date($impact_date)).days as days_diff
    
    RETURN DISTINCT
           event.id as event_id,
           labels(event)[0] as event_type,
           type(rel) as relationship_type,
           event.created as event_time,
           check_date.date as event_date,
           event.market_session as market_session,
           days_diff as days_to_impact,
           
           // Event headline
           CASE
             WHEN event:News THEN event.title
             WHEN event:Transcript THEN coalesce(event.company_name, '') + ' Q' + coalesce(event.fiscal_quarter, '') + ' ' + coalesce(toString(event.fiscal_year), '') + ' Earnings Call'
             WHEN event:Report THEN event.formType + ': ' + coalesce(event.description, '')
           END as headline,
           
           // Rich content for LangGraph (removed non-existent summary property)
           CASE
             WHEN event:News THEN substring(coalesce(event.content, ''), 0, 500)
             WHEN event:Transcript THEN 'Earnings call for ' + event.company_name + ' fiscal Q' + event.fiscal_quarter + ' ' + toString(event.fiscal_year)
             WHEN event:Report THEN 'SEC Filing: ' + event.formType + ' - ' + coalesce(event.description, '')
           END as content_preview,
           
           // Source (simplified without non-existent properties)
           CASE
             WHEN event:News THEN 'News'
             WHEN event:Transcript THEN 'Earnings Call'
             WHEN event:Report THEN 'SEC Filing'
           END as source,
           
           // URL (all event types have this)
           event.url as url,
           
           // Event-specific properties (only existing properties)
           CASE
             WHEN event:News THEN {}
             WHEN event:Transcript THEN {fiscal_year: event.fiscal_year, fiscal_quarter: event.fiscal_quarter}
             WHEN event:Report THEN {form_type: event.formType, accession_no: event.accessionNo}
           END as metadata,
           
           // Returns from relationship
           rel.hourly_stock as hourly_return,
           rel.hourly_macro as hourly_market,
           round(coalesce(rel.hourly_stock, 0) - coalesce(rel.hourly_macro, 0), 2) as hourly_adjusted,
           
           rel.session_stock as session_return,
           rel.session_macro as session_market,
           round(coalesce(rel.session_stock, 0) - coalesce(rel.session_macro, 0), 2) as session_adjusted,
           
           rel.daily_stock as daily_return,
           rel.daily_macro as daily_market,
           round(coalesce(rel.daily_stock, 0) - coalesce(rel.daily_macro, 0), 2) as daily_adjusted,
           
           // Which thresholds were met
           CASE
             WHEN abs(coalesce(rel.hourly_stock, 0) - coalesce(rel.hourly_macro, 0)) >= $hourly_threshold
             THEN 'hourly'
             ELSE null
           END as met_hourly,
           CASE
             WHEN abs(coalesce(rel.session_stock, 0) - coalesce(rel.session_macro, 0)) >= $session_threshold
             THEN 'session'
             ELSE null
           END as met_session,
           CASE
             WHEN abs(coalesce(rel.daily_stock, 0) - coalesce(rel.daily_macro, 0)) >= $daily_threshold
             THEN 'daily'
             ELSE null
           END as met_daily
           
    ORDER BY event.created
    """
    
    results = manager.execute_cypher_query_all(
        events_query,
        parameters={
            "ticker": ticker,
            "impact_date": impact_date,
            "hourly_threshold": hourly_threshold,
            "session_threshold": session_threshold,
            "daily_threshold": daily_threshold
        }
    )
    
    df = pd.DataFrame(results)
    
    if debug and not df.empty:
        print(f"\nFound {len(df)} events contributing to {impact_date}")
        threshold_summary = []
        if df['met_hourly'].notna().any():
            threshold_summary.append(f"{df['met_hourly'].notna().sum()} met hourly")
        if df['met_session'].notna().any():
            threshold_summary.append(f"{df['met_session'].notna().sum()} met session")
        if df['met_daily'].notna().any():
            threshold_summary.append(f"{df['met_daily'].notna().sum()} met daily")
        print(f"Threshold summary: {', '.join(threshold_summary)}")
    
    return df

In [4]:
def prepare_attribution_package(ticker: str, impact_date: str, 
                               hourly_threshold: float = 2.0,
                               session_threshold: float = 2.0,
                               daily_threshold: float = 2.0,
                               include_market_context: bool = True):
    """
    Prepare complete attribution data package for LangGraph agents.
    
    Parameters:
    -----------
    ticker: Company ticker
    impact_date: The date to analyze
    hourly_threshold: Min adjusted return for hourly (default 2%)
    session_threshold: Min adjusted return for session (default 2%)
    daily_threshold: Min adjusted return for daily (default 2%)
    include_market_context: Include broader market context (default True)
    
    Returns:
    --------
    Dictionary with:
    - Company context and metadata
    - Actual day's returns (4pm-to-4pm)
    - All contributing events with rich text and metadata
    - Market context for the period
    - Summary statistics
    - Ready for JSON serialization
    """
    
    manager = get_manager()
    
    # Get company context (removed non-existent marketCap)
    company_query = """
    MATCH (c:Company {ticker: $ticker})
    RETURN c.name as company_name,
           c.ticker as ticker,
           c.sector as sector,
           c.industry as industry,
           c.exchange as exchange
    """
    
    company_results = manager.execute_cypher_query_all(
        company_query,
        parameters={"ticker": ticker}
    )
    
    company_info = company_results[0] if company_results else {}
    
    # Get the actual day's return (removed non-existent market session times)
    day_query = """
    MATCH (d:Date {date: $impact_date})-[hp:HAS_PRICE]->(c:Company {ticker: $ticker})
    MATCH (d)-[hp_spy:HAS_PRICE]->(spy:MarketIndex {ticker: 'SPY'})
    
    // Get previous day for context
    OPTIONAL MATCH (prev:Date {next_trading_date: $impact_date})-[hp_prev:HAS_PRICE]->(c)
    
    RETURN d.date as date,
           hp.daily_return as stock_return,
           hp_spy.daily_return as market_return,
           round(hp.daily_return - hp_spy.daily_return, 2) as adjusted_return,
           hp.open as open_price,
           hp.close as close_price,
           hp.high as high_price,
           hp.low as low_price,
           hp.volume as volume,
           hp_prev.close as prev_close
    """
    
    day_results = manager.execute_cypher_query_all(
        day_query,
        parameters={"ticker": ticker, "impact_date": impact_date}
    )
    
    if not day_results:
        return {"error": f"No price data found for {ticker} on {impact_date}"}
    
    day_data = day_results[0]
    
    # Calculate day of week and additional metrics
    day_of_week = pd.to_datetime(impact_date).day_name()
    
    # Get contributing events with all thresholds
    events_df = get_attribution_events(ticker, impact_date, 
                                     hourly_threshold=hourly_threshold,
                                     session_threshold=session_threshold,
                                     daily_threshold=daily_threshold)
    
    # Get market context if requested
    market_context = {}
    if include_market_context:
        context_query = """
        MATCH (d:Date {date: $impact_date})-[hp:HAS_PRICE]->(idx:MarketIndex)
        WHERE idx.ticker IN ['SPY', 'QQQ', 'DIA', 'VIX']
        RETURN idx.ticker as index_ticker,
               idx.name as index_name,
               hp.daily_return as return,
               hp.volume as volume
        ORDER BY idx.ticker
        """
        
        context_results = manager.execute_cypher_query_all(
            context_query,
            parameters={"impact_date": impact_date}
        )
        
        if context_results:
            market_context = {r['index_ticker']: {
                'name': r['index_name'],
                'return': r['return'],
                'volume': r['volume']
            } for r in context_results}
    
    # Build the package
    package = {
        "attribution_analysis": {
            "company": {
                "ticker": ticker,
                "name": company_info.get('company_name', ticker),
                "sector": company_info.get('sector'),
                "industry": company_info.get('industry'),
                "exchange": company_info.get('exchange')
            },
            
            "impact_date": impact_date,
            "day_of_week": day_of_week,
            
            "price_action": {
                "open": day_data['open_price'],
                "high": day_data['high_price'],
                "low": day_data['low_price'],
                "close": day_data['close_price'],
                "prev_close": day_data['prev_close'],
                "volume": day_data['volume'],
                "dollar_change": round(day_data['close_price'] - day_data['open_price'], 2) if day_data['close_price'] and day_data['open_price'] else None,
                "intraday_range": round(day_data['high_price'] - day_data['low_price'], 2) if day_data['high_price'] and day_data['low_price'] else None
            },
            
            "returns": {
                "stock_return": day_data['stock_return'],
                "market_return": day_data['market_return'],
                "adjusted_return": day_data['adjusted_return'],
                "outperformance": round(day_data['stock_return'] - day_data['market_return'], 2)
            },
            
            "market_context": market_context,
            
            "thresholds_used": {
                "hourly": hourly_threshold,
                "session": session_threshold,
                "daily": daily_threshold
            },
            
            "contributing_events": [],
            
            "summary": {
                "total_events": 0,
                "by_type": {},
                "by_session": {},
                "by_timing": {},
                "by_threshold_met": {},
                "by_source": {}
            }
        }
    }
    
    # Process events if any
    if not events_df.empty:
        # Track various aggregations
        threshold_counts = {
            'hourly_only': 0,
            'session_only': 0,
            'daily_only': 0,
            'multiple': 0
        }
        source_counts = {}
        
        # Add events with rich content
        for _, event in events_df.iterrows():
            # Determine which thresholds were met
            met_thresholds = []
            if pd.notna(event['met_hourly']):
                met_thresholds.append('hourly')
            if pd.notna(event['met_session']):
                met_thresholds.append('session')
            if pd.notna(event['met_daily']):
                met_thresholds.append('daily')
            
            # Count threshold patterns
            if len(met_thresholds) > 1:
                threshold_counts['multiple'] += 1
            elif 'hourly' in met_thresholds:
                threshold_counts['hourly_only'] += 1
            elif 'session' in met_thresholds:
                threshold_counts['session_only'] += 1
            elif 'daily' in met_thresholds:
                threshold_counts['daily_only'] += 1
            
            # Track sources
            source = event.get('source', 'Unknown')
            source_counts[source] = source_counts.get(source, 0) + 1
            
            # Build rich event data for LangGraph
            event_data = {
                "event_id": event['event_id'],
                "event_type": event['event_type'],
                "event_datetime": str(event['event_time']),
                "event_date": event['event_date'],
                "market_session": event['market_session'],
                "days_before_impact": int(event['days_to_impact']),
                
                # Rich text content
                "headline": event['headline'],
                "content_preview": event.get('content_preview', ''),
                "source": event.get('source', ''),
                "url": event.get('url', ''),
                
                # Event-specific metadata (simplified)
                "metadata": event.get('metadata', {}),
                
                # Timing context
                "timing_context": {
                    "session": event['market_session'],
                    "days_before": abs(int(event['days_to_impact'])),
                    "is_same_day": event['days_to_impact'] == 0,
                    "is_overnight": event['days_to_impact'] < 0 and event['market_session'] == 'post_market'
                },
                
                # Impact metrics
                "immediate_impact": {
                    "hourly": {
                        "stock": event['hourly_return'],
                        "market": event['hourly_market'],
                        "adjusted": event['hourly_adjusted'],
                        "exceeded_threshold": pd.notna(event['met_hourly'])
                    },
                    "session": {
                        "stock": event['session_return'],
                        "market": event['session_market'],
                        "adjusted": event['session_adjusted'],
                        "exceeded_threshold": pd.notna(event['met_session'])
                    },
                    "daily": {
                        "stock": event['daily_return'],
                        "market": event['daily_market'],
                        "adjusted": event['daily_adjusted'],
                        "exceeded_threshold": pd.notna(event['met_daily'])
                    }
                },
                
                "thresholds_met": met_thresholds,
                
                # Scoring hints for agents
                "impact_strength": max(
                    abs(event['hourly_adjusted'] or 0),
                    abs(event['session_adjusted'] or 0),
                    abs(event['daily_adjusted'] or 0)
                ),
                "primary_impact_window": "hourly" if abs(event['hourly_adjusted'] or 0) == max(
                    abs(event['hourly_adjusted'] or 0),
                    abs(event['session_adjusted'] or 0),
                    abs(event['daily_adjusted'] or 0)
                ) else "session" if abs(event['session_adjusted'] or 0) == max(
                    abs(event['hourly_adjusted'] or 0),
                    abs(event['session_adjusted'] or 0),
                    abs(event['daily_adjusted'] or 0)
                ) else "daily"
            }
            
            package["attribution_analysis"]["contributing_events"].append(event_data)
        
        # Update summary with richer insights
        package["attribution_analysis"]["summary"].update({
            "total_events": len(events_df),
            "by_type": events_df['event_type'].value_counts().to_dict(),
            "by_session": events_df['market_session'].value_counts().to_dict(),
            "by_timing": events_df.groupby('days_to_impact').size().to_dict(),
            "by_threshold_met": {k: v for k, v in threshold_counts.items() if v > 0},
            "by_source": source_counts
        })
        
        # Add strongest impact event with more context
        events_df['max_impact'] = events_df[['hourly_adjusted', 'session_adjusted', 'daily_adjusted']].abs().max(axis=1)
        strongest_idx = events_df['max_impact'].idxmax()
        if pd.notna(strongest_idx):
            strongest = events_df.loc[strongest_idx]
            package["attribution_analysis"]["summary"]["strongest_immediate_impact"] = {
                "event_id": strongest['event_id'],
                "event_type": strongest['event_type'],
                "headline": strongest['headline'],
                "content_preview": strongest.get('content_preview', '')[:200] + '...' if strongest.get('content_preview') else '',
                "max_impact": strongest['max_impact'],
                "impact_type": "hourly" if abs(strongest['hourly_adjusted'] or 0) == strongest['max_impact'] 
                              else "session" if abs(strongest['session_adjusted'] or 0) == strongest['max_impact']
                              else "daily",
                "source": strongest.get('source', '')
            }
        
        # Add timing insights
        pre_market_events = [e for e in package["attribution_analysis"]["contributing_events"] if e['market_session'] == 'pre_market']
        post_market_events = [e for e in package["attribution_analysis"]["contributing_events"] if e['timing_context']['is_overnight']]
        
        package["attribution_analysis"]["summary"]["timing_insights"] = {
            "overnight_catalyst_count": len(post_market_events),
            "pre_market_reaction_count": len(pre_market_events),
            "has_earnings_event": any(e['event_type'] == 'Transcript' for e in package["attribution_analysis"]["contributing_events"]),
            "has_sec_filing": any(e['event_type'] == 'Report' for e in package["attribution_analysis"]["contributing_events"])
        }
    
    return package

## Utility Functions

In [5]:
def prepare_attribution_references(ticker: str, impact_date: str, 
                                 hourly_threshold: float = 2.0,
                                 session_threshold: float = 2.0,
                                 daily_threshold: float = 2.0):
    """
    Prepare lightweight attribution package with node references for LangGraph agents.
    Agents can use MCP to fetch full details for any referenced nodes.
    
    Parameters:
    -----------
    ticker: Company ticker
    impact_date: The date to analyze
    hourly_threshold: Min adjusted return for hourly (default 2%)
    session_threshold: Min adjusted return for session (default 2%)
    daily_threshold: Min adjusted return for daily (default 2%)
    
    Returns:
    --------
    Dictionary with:
    - Core metrics and returns
    - Event references with key properties
    - Cypher queries for agents to fetch more data
    - Node IDs for direct MCP access
    """
    
    manager = get_manager()
    
    # Get the actual day's return and node references
    day_query = """
    MATCH (d:Date {date: $impact_date})-[hp:HAS_PRICE]->(c:Company {ticker: $ticker})
    MATCH (d)-[hp_spy:HAS_PRICE]->(spy:MarketIndex {ticker: 'SPY'})
    
    RETURN // Core data
           d.date as date,
           hp.daily_return as stock_return,
           hp_spy.daily_return as market_return,
           round(hp.daily_return - hp_spy.daily_return, 2) as adjusted_return,
           hp.volume as volume,
           
           // Node references for MCP
           id(d) as date_node_id,
           id(c) as company_node_id,
           id(hp) as price_relationship_id,
           
           // Key identifiers
           c.ticker as ticker,
           c.name as company_name,
           c.sector as sector
    """
    
    day_results = manager.execute_cypher_query_all(
        day_query,
        parameters={"ticker": ticker, "impact_date": impact_date}
    )
    
    if not day_results:
        return {"error": f"No price data found for {ticker} on {impact_date}"}
    
    day_data = day_results[0]
    day_of_week = pd.to_datetime(impact_date).day_name()
    
    # Get contributing events with minimal data (fixed ORDER BY)
    events_query = """
    MATCH (impact:Date {date: $impact_date})
    MATCH (c:Company {ticker: $ticker})
    
    // Get previous trading days
    OPTIONAL MATCH path = (d1:Date)-[:NEXT*1..5]->(impact)
    WHERE d1.is_trading_day = '1'
    
    WITH impact, c, collect(DISTINCT d1) + [impact] as check_dates
    
    // Find events on these dates
    UNWIND check_dates as check_date
    MATCH (event)-[rel:INFLUENCES|PRIMARY_FILER]->(c)
    WHERE (event:News OR event:Transcript OR event:Report)
      AND date(datetime(event.created)) = date(check_date.date)
      AND (
        (check_date.date < $impact_date AND event.market_session = 'post_market')
        OR
        (check_date.date = $impact_date AND event.market_session IN ['pre_market', 'in_market'])
      )
      AND (
        abs(coalesce(rel.hourly_stock, 0) - coalesce(rel.hourly_macro, 0)) >= $hourly_threshold
        OR abs(coalesce(rel.session_stock, 0) - coalesce(rel.session_macro, 0)) >= $session_threshold
        OR abs(coalesce(rel.daily_stock, 0) - coalesce(rel.daily_macro, 0)) >= $daily_threshold
      )
    
    WITH event, rel, check_date, impact, c,
         duration.between(date(check_date.date), date($impact_date)).days as days_diff,
         round(coalesce(rel.hourly_stock, 0) - coalesce(rel.hourly_macro, 0), 2) as hourly_adj,
         round(coalesce(rel.session_stock, 0) - coalesce(rel.session_macro, 0), 2) as session_adj,
         round(coalesce(rel.daily_stock, 0) - coalesce(rel.daily_macro, 0), 2) as daily_adj
    
    RETURN DISTINCT
           // Node references
           id(event) as event_node_id,
           event.id as event_id,
           labels(event)[0] as event_type,
           id(rel) as relationship_id,
           type(rel) as relationship_type,
           
           // Minimal key data
           event.created as event_time,
           event.market_session as market_session,
           days_diff as days_to_impact,
           
           // Just the headline for context
           CASE
             WHEN event:News THEN event.title
             WHEN event:Transcript THEN coalesce(event.company_name, '') + ' Earnings Call'
             WHEN event:Report THEN event.formType + ' Filing'
           END as headline,
           
           // Key impact metrics only
           hourly_adj as hourly_adjusted,
           session_adj as session_adjusted,
           daily_adj as daily_adjusted,
           
           // Which thresholds were met
           CASE
             WHEN abs(hourly_adj) >= $hourly_threshold THEN true
             ELSE false
           END as met_hourly,
           CASE
             WHEN abs(session_adj) >= $session_threshold THEN true
             ELSE false
           END as met_session,
           CASE
             WHEN abs(daily_adj) >= $daily_threshold THEN true
             ELSE false
           END as met_daily,
           
           // Max impact for sorting
           CASE
             WHEN abs(hourly_adj) >= abs(session_adj) AND abs(hourly_adj) >= abs(daily_adj)
             THEN abs(hourly_adj)
             WHEN abs(session_adj) >= abs(daily_adj)
             THEN abs(session_adj)
             ELSE abs(daily_adj)
           END as max_impact
           
    ORDER BY max_impact DESC
    """
    
    events_results = manager.execute_cypher_query_all(
        events_query,
        parameters={
            "ticker": ticker,
            "impact_date": impact_date,
            "hourly_threshold": hourly_threshold,
            "session_threshold": session_threshold,
            "daily_threshold": daily_threshold
        }
    )
    
    # Build lightweight package with references
    package = {
        "attribution_references": {
            "query_context": {
                "ticker": ticker,
                "impact_date": impact_date,
                "day_of_week": day_of_week,
                "thresholds": {
                    "hourly": hourly_threshold,
                    "session": session_threshold,
                    "daily": daily_threshold
                }
            },
            
            "day_summary": {
                "stock_return": day_data['stock_return'],
                "market_return": day_data['market_return'],
                "adjusted_return": day_data['adjusted_return'],
                "volume": day_data['volume'],
                "company_name": day_data['company_name'],
                "sector": day_data['sector']
            },
            
            "node_references": {
                "date_node_id": day_data['date_node_id'],
                "company_node_id": day_data['company_node_id'],
                "price_relationship_id": day_data['price_relationship_id']
            },
            
            "event_references": [],
            
            "mcp_helper_queries": {
                "fetch_full_event": "MATCH (e) WHERE id(e) = $node_id RETURN e",
                "fetch_event_content": "MATCH (e:News|Transcript|Report) WHERE id(e) = $node_id RETURN e.content as content, e.summary as summary",
                "fetch_related_events": "MATCH (e1)-[:INFLUENCES|PRIMARY_FILER]->(c:Company {ticker: $ticker})<-[:INFLUENCES|PRIMARY_FILER]-(e2) WHERE id(e1) = $node_id AND date(datetime(e2.created)) = date(datetime(e1.created)) RETURN e2",
                "fetch_price_details": "MATCH ()-[hp:HAS_PRICE]->() WHERE id(hp) = $relationship_id RETURN hp",
                "fetch_market_context": "MATCH (d:Date {date: $date})-[hp:HAS_PRICE]->(idx:MarketIndex) RETURN idx.ticker as index, hp.daily_return as return"
            }
        }
    }
    
    # Add event references
    for event in events_results:
        # Determine which thresholds were met
        thresholds_met = []
        if event['met_hourly']:
            thresholds_met.append('hourly')
        if event['met_session']:
            thresholds_met.append('session')
        if event['met_daily']:
            thresholds_met.append('daily')
        
        event_ref = {
            "event_node_id": event['event_node_id'],
            "event_id": event['event_id'],
            "event_type": event['event_type'],
            "relationship_id": event['relationship_id'],
            "relationship_type": event['relationship_type'],
            
            "headline": event['headline'],
            "event_time": str(event['event_time']),
            "market_session": event['market_session'],
            "days_to_impact": event['days_to_impact'],
            
            "impact_summary": {
                "hourly_adjusted": event['hourly_adjusted'],
                "session_adjusted": event['session_adjusted'],
                "daily_adjusted": event['daily_adjusted'],
                "thresholds_met": thresholds_met,
                "max_impact": event['max_impact']
            }
        }
        
        package["attribution_references"]["event_references"].append(event_ref)
    
    # Add summary statistics
    if events_results:
        event_types = {}
        sessions = {}
        for event in events_results:
            event_types[event['event_type']] = event_types.get(event['event_type'], 0) + 1
            sessions[event['market_session']] = sessions.get(event['market_session'], 0) + 1
        
        package["attribution_references"]["summary"] = {
            "total_events": len(events_results),
            "by_type": event_types,
            "by_session": sessions,
            "strongest_impact_event_id": events_results[0]['event_node_id'] if events_results else None
        }
    else:
        package["attribution_references"]["summary"] = {
            "total_events": 0,
            "by_type": {},
            "by_session": {},
            "strongest_impact_event_id": None
        }
    
    return package

## Usage Example

In [ ]:

# Step 1: Find significant days for a ticker
print("=" * 80)
print("STEP 1: Finding significant days for AAPL")
print("=" * 80)
significant_days = find_significant_days('AAPL', limit=5000)
print(f"Found {len(significant_days)} significant days")
print("\nResults:")
print(significant_days[['date', 'day_of_week', 'stock_return', 'adjusted_return']].head())

# Sense check
print("\nSense Check:")
print(f"- All stock returns >= 2%: {(significant_days['stock_return'].abs() >= 2.0).all()}")
print(f"- All adjusted returns >= 2%: {(significant_days['adjusted_return'].abs() >= 2.0).all()}")
print(f"- Date range: {significant_days['date'].min()} to {significant_days['date'].max()}")

STEP 1: Finding significant days for AAPL
Found 43 significant days

Results:
         date day_of_week  stock_return  adjusted_return
0  2024-06-11     Tuesday          7.26             7.02
1  2025-05-02      Friday         -3.74            -5.22
2  2025-04-09   Wednesday         15.33             4.83
3  2024-05-03      Friday          5.98             4.74
4  2025-01-27      Monday          3.18             4.59

Sense Check:
- All stock returns >= 2%: True
- All adjusted returns >= 2%: True
- Date range: 2023-01-03 to 2025-06-20


In [7]:


# Step 2a: Get FULL attribution package (for standalone use)
if not significant_days.empty:
    test_date = significant_days.iloc[0]['date']
    print(f"\n{'=' * 80}")
    print(f"STEP 2A: Full Attribution Package for {test_date}")
    print("=" * 80)
    
    full_package = prepare_attribution_package('AAPL', test_date)
    
    print(f"Total events: {full_package['attribution_analysis']['summary']['total_events']}")
    print(f"Company: {full_package['attribution_analysis']['company']['name']}")
    print(f"Sector: {full_package['attribution_analysis']['company']['sector']}")
    print(f"Market context included: {'market_context' in full_package['attribution_analysis']}")
    
    # Show price action
    price = full_package['attribution_analysis']['price_action']
    print(f"\nPrice Action on {test_date}:")
    print(f"- Open: ${price['open']:.2f}, Close: ${price['close']:.2f}")
    print(f"- High: ${price['high']:.2f}, Low: ${price['low']:.2f}")
    print(f"- Volume: {price['volume']:,}")
    
    # Show returns
    returns = full_package['attribution_analysis']['returns']
    print(f"\nReturns:")
    print(f"- Stock: {returns['stock_return']}%")
    print(f"- Market: {returns['market_return']}%")
    print(f"- Adjusted: {returns['adjusted_return']}%")
    
    # Show events summary
    summary = full_package['attribution_analysis']['summary']
    print(f"\nEvent Summary:")
    print(f"- By type: {summary['by_type']}")
    print(f"- By session: {summary['by_session']}")
    print(f"- By source: {summary.get('by_source', {})}")
    
    # Show timing insights
    if 'timing_insights' in summary:
        timing = summary['timing_insights']
        print(f"\nTiming Insights:")
        print(f"- Overnight catalysts: {timing['overnight_catalyst_count']}")
        print(f"- Pre-market reactions: {timing['pre_market_reaction_count']}")
        print(f"- Has earnings: {timing['has_earnings_event']}")
        print(f"- Has SEC filing: {timing['has_sec_filing']}")
    
    # Show sample event with rich content
    if full_package['attribution_analysis']['contributing_events']:
        sample_event = full_package['attribution_analysis']['contributing_events'][0]
        print(f"\nSample Event:")
        print(f"- Type: {sample_event['event_type']}")
        print(f"- Headline: {sample_event['headline'][:80]}...")
        print(f"- Content preview: {sample_event['content_preview'][:100]}..." if sample_event['content_preview'] else "- Content preview: None")
        print(f"- Source: {sample_event['source']}")
        print(f"- Session: {sample_event['market_session']}")
        print(f"- Impact strength: {sample_event['impact_strength']}%")


STEP 2A: Full Attribution Package for 2024-06-11
Total events: 15
Company: APPLE INC
Sector: Technology
Market context included: True

Price Action on 2024-06-11:
- Open: $193.65, Close: $207.15
- High: $207.16, Low: $193.63
- Volume: 172,116,296.0

Returns:
- Stock: 7.26%
- Market: 0.24%
- Adjusted: 7.02%

Event Summary:
- By type: {'News': 15}
- By session: {'pre_market': 8, 'in_market': 5, 'post_market': 2}
- By source: {'News': 15}

Timing Insights:
- Overnight catalysts: 0
- Pre-market reactions: 8
- Has earnings: False
- Has SEC filing: False

Sample Event:
- Type: News
- Headline: Apple Unveils Suite Of New Tools And Resources Designed To Enable Developers To ...
- Content preview: None
- Source: News
- Session: post_market
- Impact strength: 7.04%


In [8]:
# Example usage - RUNNING ALL EXAMPLES


# Step 2b: Get LIGHTWEIGHT references (for LangGraph + MCP)
if not significant_days.empty:
    test_date = significant_days.iloc[0]['date']
    print(f"\n{'=' * 80}")
    print(f"STEP 2B: Lightweight References for {test_date}")
    print("=" * 80)
    
    ref_package = prepare_attribution_references('AAPL', test_date)
    
    print(f"Total events: {ref_package['attribution_references']['summary']['total_events']}")
    
    # Show node references
    nodes = ref_package['attribution_references']['node_references']
    print(f"\nNode IDs provided:")
    print(f"- Date node: {nodes['date_node_id']}")
    print(f"- Company node: {nodes['company_node_id']}")
    print(f"- Price relationship: {nodes['price_relationship_id']}")
    
    # Show MCP queries
    print(f"\nMCP queries available:")
    for query_name in ref_package['attribution_references']['mcp_helper_queries'].keys():
        print(f"- {query_name}")
    
    # Show sample event reference
    if ref_package['attribution_references']['event_references']:
        sample_ref = ref_package['attribution_references']['event_references'][0]
        print(f"\nSample Event Reference:")
        print(f"- Node ID: {sample_ref['event_node_id']}")
        print(f"- Event ID: {sample_ref['event_id']}")
        print(f"- Type: {sample_ref['event_type']}")
        print(f"- Headline: {sample_ref['headline']}")
        print(f"- Max impact: {sample_ref['impact_summary']['max_impact']}%")
        print(f"- Thresholds met: {sample_ref['impact_summary']['thresholds_met']}")



STEP 2B: Lightweight References for 2024-06-11


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 13, column: 12, offset: 476} for query: "\n    MATCH (d:Date {date: $impact_date})-[hp:HAS_PRICE]->(c:Company {ticker: $ticker})\n    MATCH (d)-[hp_spy:HAS_PRICE]->(spy:MarketIndex {ticker: 'SPY'})\n    \n    RETURN // Core data\n           d.date as date,\n           hp.daily_return as stock_return,\n           hp_spy.daily_return as market_return,\n           round(hp.daily_return - hp_spy.daily_return, 2) as adjusted_return,\n           hp.volume as volume,\n           \n           // Node references for MCP\n           id(d) as date_node_id,\n           id(c) as company_node_id,\n           id(hp) as price_relationship_id,\n           \n           // Key identifiers\n           c.tick

Total events: 15

Node IDs provided:
- Date node: 3314
- Company node: 2280
- Price relationship: 1729388853980040434

MCP queries available:
- fetch_full_event
- fetch_event_content
- fetch_related_events
- fetch_price_details
- fetch_market_context

Sample Event Reference:
- Node ID: 6114269
- Event ID: bzNews_39260124
- Type: News
- Headline: Apple Unveils Suite Of New Tools And Resources Designed To Enable Developers To Create Powerful, Efficient Apps Across All Apple Platforms
- Max impact: 7.04%
- Thresholds met: ['session', 'daily']


In [9]:

# Example MCP usage simulation
print(f"\n{'=' * 80}")
print("STEP 3: MCP Usage Example (Simulated)")
print("=" * 80)
print("# 1. Get references: refs = prepare_attribution_references('AAPL', '2024-06-11')")
print("# 2. Pick an event: event_node_id = refs['attribution_references']['event_references'][0]['event_node_id']")
print("# 3. Use MCP to fetch full content:") 
print("#    query = refs['attribution_references']['mcp_helper_queries']['fetch_event_content']")
print("#    content = mcp_execute(query, {'node_id': event_node_id})")



STEP 3: MCP Usage Example (Simulated)
# 1. Get references: refs = prepare_attribution_references('AAPL', '2024-06-11')
# 2. Pick an event: event_node_id = refs['attribution_references']['event_references'][0]['event_node_id']
# 3. Use MCP to fetch full content:
#    query = refs['attribution_references']['mcp_helper_queries']['fetch_event_content']
#    content = mcp_execute(query, {'node_id': event_node_id})


In [10]:

# Test with different ticker
print(f"\n{'=' * 80}")
print("STEP 4: Testing with different ticker (NVDA)")
print("=" * 80)
nvda_days = find_significant_days('NVDA', start_date='2024-01-01', limit=3)
print(f"Found {len(nvda_days)} significant days for NVDA")
if not nvda_days.empty:
    print(nvda_days[['date', 'stock_return', 'adjusted_return']])
    
    # Get attribution for NVDA
    nvda_date = nvda_days.iloc[0]['date']
    nvda_refs = prepare_attribution_references('NVDA', nvda_date)
    print(f"\nAttribution for NVDA on {nvda_date}:")
    print(f"- Events found: {nvda_refs['attribution_references']['summary']['total_events']}")
    print(f"- By type: {nvda_refs['attribution_references']['summary']['by_type']}")
    print(f"- By session: {nvda_refs['attribution_references']['summary']['by_session']}")

print("\n" + "=" * 80)
print("ALL EXAMPLES COMPLETED SUCCESSFULLY")
print("=" * 80)


STEP 4: Testing with different ticker (NVDA)
Found 3 significant days for NVDA
         date  stock_return  adjusted_return
0  2025-01-27        -16.97           -15.56
1  2024-02-22         16.40            14.33
2  2024-07-31         12.81            11.18


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 13, column: 12, offset: 476} for query: "\n    MATCH (d:Date {date: $impact_date})-[hp:HAS_PRICE]->(c:Company {ticker: $ticker})\n    MATCH (d)-[hp_spy:HAS_PRICE]->(spy:MarketIndex {ticker: 'SPY'})\n    \n    RETURN // Core data\n           d.date as date,\n           hp.daily_return as stock_return,\n           hp_spy.daily_return as market_return,\n           round(hp.daily_return - hp_spy.daily_return, 2) as adjusted_return,\n           hp.volume as volume,\n           \n           // Node references for MCP\n           id(d) as date_node_id,\n           id(c) as company_node_id,\n           id(hp) as price_relationship_id,\n           \n           // Key identifiers\n           c.tick


Attribution for NVDA on 2025-01-27:
- Events found: 5
- By type: {'News': 5}
- By session: {'in_market': 4, 'pre_market': 1}

ALL EXAMPLES COMPLETED SUCCESSFULLY


In [11]:

# Step 2a: Get FULL attribution package (for standalone use)
if not significant_days.empty:
    test_date = significant_days.iloc[0]['date']
    full_package = prepare_attribution_package('AAPL', test_date)
    print(f"\nFull Attribution for {test_date}:")
    print(f"Total events: {full_package['attribution_analysis']['summary']['total_events']}")
    print(f"Company: {full_package['attribution_analysis']['company']['name']}")
    print(f"Market context included: {'market_context' in full_package['attribution_analysis']}")


Full Attribution for 2024-06-11:
Total events: 15
Company: APPLE INC
Market context included: True


In [12]:



# Step 2b: Get LIGHTWEIGHT references (for LangGraph + MCP)
if not significant_days.empty:
    test_date = significant_days.iloc[0]['date']
    ref_package = prepare_attribution_references('AAPL', test_date)
    print(f"\nLightweight References for {test_date}:")
    print(f"Total events: {ref_package['attribution_references']['summary']['total_events']}")
    print(f"Node IDs provided: {ref_package['attribution_references']['node_references']}")
    print(f"MCP queries available: {list(ref_package['attribution_references']['mcp_helper_queries'].keys())}")



Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 13, column: 12, offset: 476} for query: "\n    MATCH (d:Date {date: $impact_date})-[hp:HAS_PRICE]->(c:Company {ticker: $ticker})\n    MATCH (d)-[hp_spy:HAS_PRICE]->(spy:MarketIndex {ticker: 'SPY'})\n    \n    RETURN // Core data\n           d.date as date,\n           hp.daily_return as stock_return,\n           hp_spy.daily_return as market_return,\n           round(hp.daily_return - hp_spy.daily_return, 2) as adjusted_return,\n           hp.volume as volume,\n           \n           // Node references for MCP\n           id(d) as date_node_id,\n           id(c) as company_node_id,\n           id(hp) as price_relationship_id,\n           \n           // Key identifiers\n           c.tick


Lightweight References for 2024-06-11:
Total events: 15
Node IDs provided: {'date_node_id': 3314, 'company_node_id': 2280, 'price_relationship_id': 1729388853980040434}
MCP queries available: ['fetch_full_event', 'fetch_event_content', 'fetch_related_events', 'fetch_price_details', 'fetch_market_context']


Example MCP usage by LangGraph agent:
```python
# 1. Get references: refs = prepare_attribution_references('AAPL', '2024-06-11')
# 2. Pick an event: event_node_id = refs['attribution_references']['event_references'][0]['event_node_id']
# 3. Use MCP to fetch full content: 
#    query = refs['attribution_references']['mcp_helper_queries']['fetch_event_content']
#    content = mcp_execute(query, {'node_id': event_node_id})
```